<a href="https://colab.research.google.com/github/yoohw0317/MILAB/blob/main/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# 통일성을 위해 시드 설정 
torch.manual_seed(1)

In [3]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [4]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [5]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [6]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


In [7]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

hypothesis 를 나타내는 것은 sigmoid 함수

In [8]:
hypothesis = 1 / (1+torch.exp(-(x_train.matmul(W) +  b))) # torch.matmul(x, w)로도 쓸 수 있음

In [9]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


In [10]:
print ('1/(1+e^{-1}) equals: ', torch.sigmoid(torch.FloatTensor([1])))

1/(1+e^{-1}) equals:  tensor([0.7311])


In [11]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [12]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])


For one element, the loss can be computed as follows:

In [13]:
- (y_train[0] * torch.log(hypothesis[0]) +  (1 - y_train[0]) * torch.log(1-hypothesis[0]))

tensor([0.6931], grad_fn=<NegBackward0>)

To compute the losses for the entire batch, we can simply input the entire vector

In [14]:
losses = - (y_train[0] * torch.log(hypothesis[0]) +  (1 - y_train[0]) * torch.log(1-hypothesis[0]))
print(losses)

tensor([0.6931], grad_fn=<NegBackward0>)


Then, we just .mean() to take the mean of these individual losses.

In [15]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [16]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

In [17]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=1)
nb_epochs= 1000
for epoch in range(nb_epochs + 1):
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis , y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031672
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


# Evaluation

After we finish training the model, we want to check how well our model fits the training set.

In [18]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
torch.set_printoptions(sci_mode = False, precision = 4)
print(hypothesis)

tensor([[    0.0003],
        [    0.0316],
        [    0.0390],
        [    0.9562],
        [    0.9982],
        [    0.9997]], grad_fn=<SigmoidBackward0>)


In [19]:
prediction = (hypothesis >= torch.FloatTensor([0.5])).float()
print(prediction)

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [20]:
print(prediction)
print(y_train)

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [21]:
correct_prediction = prediction == y_train
print (correct_prediction)

tensor([[True],
        [True],
        [True],
        [True],
        [True],
        [True]])


Higher Implementation with Class

In [22]:
class BinaryClassifier(nn.Module):
    def __init__ (self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [23]:
model = BinaryClassifier()

In [24]:
optimizer = optim.SGD(model.parameters(), lr = 1)
nb_epochs = 100
for epoch in range (nb_epochs + 1):
    hypothesis = model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len (correct_prediction)
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}, Accuracy {accuracy * 100: 2.2f}')

Epoch    0/100 Cost: 0.539713, Accuracy  83.33
Epoch   10/100 Cost: 0.614853, Accuracy  66.67
Epoch   20/100 Cost: 0.441875, Accuracy  66.67
Epoch   30/100 Cost: 0.373145, Accuracy  83.33
Epoch   40/100 Cost: 0.316358, Accuracy  83.33
Epoch   50/100 Cost: 0.266094, Accuracy  83.33
Epoch   60/100 Cost: 0.220498, Accuracy  100.00
Epoch   70/100 Cost: 0.182095, Accuracy  100.00
Epoch   80/100 Cost: 0.157299, Accuracy  100.00
Epoch   90/100 Cost: 0.144091, Accuracy  100.00
Epoch  100/100 Cost: 0.134272, Accuracy  100.00
